<a href="https://colab.research.google.com/github/JeongSeong/brainML_HW/blob/main/Neural_Networks_for_Multi_task_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

**Load TADPOLE* dataset (csv file) from Google Drive**
-------------------------------------------------------
*The Alzheimer's Disease Prediction Of Longitudinal Evolution
(https://tadpole.grand-challenge.org/)

### -Subjects: 1707 (1363 Train (80%) + 344 Test (20%))


### -Features: 72
*   2 demographic feature: MMSE, ADAS13
*   70 mean values of cortical thickness

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

csv_file_train = '/content/gdrive/My Drive/BNCS401_Midterm_Project/Train_data_reupdated.csv'  # Set your path
train_data = pd.read_csv(csv_file_train)
train_data

# DXCHANGE: clinical label (1-CN, 2-MCI, 3-AD)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,RID,DXCHANGE,AGE,MMSE,ADAS13,ST102TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST103TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST104TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST105TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST106TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST107TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST108TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST109TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST110TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST111TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST113TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST114TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST115TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST116TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST117TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST118TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST119TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST121TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST123TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST129TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST130TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST13TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST14TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST15TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST23TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST24TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST25TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST26TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST31TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST32TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST34TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST35TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST36TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST38TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST39TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST40TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST43TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST44TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST45TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST46TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST47TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST48TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST49TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST50TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST51TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST52TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST54TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST55TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST56TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST57TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST58TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST59TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST60TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST62TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST64TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST72TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST73TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST74TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST82TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST83TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST84TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST85TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST90TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST91TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST93TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST94TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST95TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST97TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST98TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16
0,4084,1,68.4,30,10.00,2.700,2.635,2.613,2.904,2.311,1.647,2.139,2.652,2.604,2.480,3.095,2.144,2.792,2.207,2.903,2.617,4.117,2.701,,3.127,3.051,2.305,2.872,2.732,2.026,3.756,2.813,2.762,2.556,2.916,2.695,2.259,2.690,2.017,2.421,2.949,2.570,2.370,2.674,3.004,2.369,1.599,2.208,2.650,2.739,2.544,3.018,2.377,2.880,2.322,2.657,2.489,3.620,2.711,,2.593,2.792,2.660,1.993,3.734,2.390,2.817,2.471,2.990,2.667,2.490,2.523,2.254,2.171,2.862
1,2196,2,68.2,30,13.00,2.453,2.992,2.470,2.965,2.438,1.584,1.910,2.900,2.451,2.335,2.771,2.354,2.712,2.001,2.729,2.363,3.613,2.475,,3.196,3.334,2.343,2.729,2.627,1.742,3.383,2.647,2.758,2.394,2.634,2.334,2.241,2.824,1.865,2.383,2.866,2.334,2.793,2.413,2.874,2.316,1.478,1.909,2.780,2.589,2.133,3.036,2.329,2.687,2.070,2.783,2.594,3.405,2.367,,2.582,2.977,2.489,1.868,3.220,2.683,2.569,2.372,2.854,2.867,2.233,2.793,1.987,2.428,2.943
2,657,1,77.7,29,15.33,2.249,2.296,2.315,2.681,2.420,1.386,1.830,2.466,2.3

#WRITE YOUR CODE HERE!!!!!!!

In [ ]:
# the result of the midterm project
for i in train_data.columns:#since there are object type data and null, it needs to be changed.
  if train_data[i].dtype=='object':#for object type data column.
    train_data[i]=pd.to_numeric(train_data[i], errors='coerce')#numeric entries are changed to float type and missing entries are replaced with NaN.
  if train_data[i].isnull().any():#missing entries are replaced with NaN.
    train_data[i].loc[train_data[i].isnull()]=np.NaN

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
train_data= SimpleImputer(strategy='median').fit_transform(train_data)#replace missing values using the median along each column.
train_columns_list=[i for i in range(len(train_data[0])) if i not in [0,1,3,4]]#a columns list for training data(without RID, DXCHANGE, MMSE, ADAS13)
x=StandardScaler().fit_transform(train_data[:, train_columns_list])#an ndarray for training data #index 699 row of the initial data order has some median data.
y=train_data[:, [3, 4]]# an ndarray for target data(MMSE, ADAS13)
dxchange=train_data[:,1]# an array for target data(DXCHANGE)

#regrerssion task
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
lin_reg=LinearRegression()#making linear regression model
lin_reg.fit(x,y)#fitting train data and target
#10 fold cross validation score
print('mean square error: ',np.sqrt(-cross_val_score(lin_reg, x, y, scoring="neg_mean_squared_error", cv=10)).mean())#since the test data is 20% of the total data, I chose k=10.

#classification task. since it took a lot of time, I just implemented the result of the grid search.
from sklearn.linear_model import SGDClassifier
sgd_clf=SGDClassifier(alpha=0.15599452033620265, average=False, class_weight=None,
              early_stopping=True, epsilon=0.1, eta0=0.1, fit_intercept=True,
              l1_ratio=0.020584494295802447, learning_rate='adaptive',
              loss='hinge', max_iter=1000, n_iter_no_change=5, n_jobs=None,
              penalty='elasticnet', power_t=0.5, random_state=None,
              shuffle=True, tol=0.001, validation_fraction=0.2, verbose=0,
              warm_start=False)
sgd_clf.fit(x,dxchange)#Avg. loss: 0.274994
print('score: ',sgd_clf.score(x, dxchange))#since the test data is 20% of the total data, I chose k=10.

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


mean square error:  5.27733401244674
score:  0.5656639765223771


In [ ]:
# the code for this assignment
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
#assert tf.__version__ >= "2.0"

In [ ]:
from tensorflow.keras.utils import to_categorical

y_mmse=y[:,0]#mmse data
y_adas13=y[:,1]#adas13 data
y_dxchange=to_categorical(dxchange)#making dxchange as on-hot vector
y_all=train_data[:,[1,3,4]]#dxchange, mmse, adas13 data
# these are already imputed and scaled.
#since there are no best way to choose hyperparameter, I just randomly tried and the below one was the best.
#I followed the recommendation of the lecture note 6
input_one=keras.layers.Input(shape=[71,])
hidden1=keras.layers.Dense(60, activation="selu", kernel_initializer="lecun_normal")(input_one)
hidden2=keras.layers.Dense(50, activation="selu", kernel_initializer="lecun_normal")(hidden1)

concat=keras.layers.concatenate([input_one, hidden2])#for wider neural network

mmse=keras.layers.Dense(1, activation="selu", kernel_initializer="lecun_normal", name="MMSE")(concat)#MMSE output
adas13=keras.layers.Dense(1, activation="selu", kernel_initializer="lecun_normal", name="ADAS13")(concat)#ADAS13 output
DC=keras.layers.Dense(4, activation="softmax", kernel_initializer="glorot_uniform", name="DXCHANGE")(concat)#dxchange ouput
aux_output1=keras.layers.Dense(2)(hidden2)#auxiliary output for the regularization technique

model=keras.models.Model(inputs=[input_one], outputs=[mmse, adas13, DC, aux_output1])#the neural network
model.compile(loss=["mse", "mse", "categorical_crossentropy", "mse"], loss_weights=[1, 1, 1, 0.1],
              optimizer="nadam", metrics={"MMSE": "mse", "ADAS13": "mse", "DXCHANGE": 'accuracy'})
es=keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)#early stopping callback
history=model.fit(#as I have done in mid term project, I used 0.2 validation data
    (x), (y_mmse, y_adas13, to_categorical(dxchange), y), epochs=1000,
    validation_split=0.2,
    callbacks=[es]
)

Train on 1090 samples, validate on 273 samples
Epoch 1/1000
1090/1090 [==============================] - 14s 13ms/sample - loss: 1053.7990 - MMSE_loss: 682.3038 - ADAS13_loss: 314.4277 - DXCHANGE_loss: 1.8053 - dense_363_loss: 558.6465 - MMSE_mean_squared_error: 675.8224 - ADAS13_mean_squared_error: 320.1769 - DXCHANGE_acc: 0.2404 - val_loss: 854.6492 - val_MMSE_loss: 558.3250 - val_ADAS13_loss: 240.6903 - val_DXCHANGE_loss: 1.4035 - val_dense_363_loss: 543.1323 - val_MMSE_mean_squared_error: 556.1771 - val_ADAS13_mean_squared_error: 242.6576 - val_DXCHANGE_acc: 0.3919
Epoch 2/1000
1090/1090 [==============================] - 1s 571us/sample - loss: 738.9459 - MMSE_loss: 467.5975 - ADAS13_loss: 223.8336 - DXCHANGE_loss: 1.2309 - dense_363_loss: 449.1541 - MMSE_mean_squared_error: 467.8206 - ADAS13_mean_squared_error: 224.3087 - DXCHANGE_acc: 0.4202 - val_loss: 694.3781 - val_MMSE_loss: 444.1051 - val_ADAS13_loss: 217.5838 - val_DXCHANGE_loss: 1.3295 - val_dense_363_loss: 336.0833 - val

In [ ]:
#I have not found the method for evaluating cross validation score for funcrional API model
#So, for the comparing, I used mean squared error and accuracy.
#However, since I made the result of regression task not separately before, I could not compare with the result of this assignment.
#Therefore, let me just explain about the result.
#MMSE and DXCHANGE results are seems to be similar to the mid-term project but, ADAS13 result is not really good.